# Spatter Benchmark Worflow

This notebook will guide you through the process of performing memory performance analysis using the Spatter benchmark. For more detail please refer to our [wiki](https://github.com/hpcgarage/spatter/wiki/Spatter-Benchmark-Workflow). As an example, we will use [Quicksilver](https://github.com/LLNL/Quicksilver) with the [All Absorb](https://github.com/LLNL/Quicksilver/blob/master/Examples/AllAbsorb/allAbsorb.inp) input.

### Installations

1) Install [Quicksilver](https://github.com/LLNL/Quicksilver)
    1) Build with OpenMP and debug symbols ON
2) Install [VTune](https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler-download.html)
3) Install [gs_patterns](https://github.com/lanl/gs_patterns)
    1) Make sure to also install [PIN](https://www.intel.com/content/www/us/en/developer/articles/tool/pin-a-dynamic-binary-instrumentation-tool.html) following the steps in the [pin_tracing](https://github.com/lanl/gs_patterns/tree/main/pin_tracing) folder

In [1]:
export nb_dir="$(pwd)"
export qs_dir="$(pwd)/Quicksilver"
export gspatterns_dir="$(pwd)/gs_patterns"

In [2]:
cd $nb_dir
#Step 1
git clone https://github.com/LLNL/Quicksilver.git
#Comment lines
sed -i '/ROCM_ROOT = /s/^/#/g' $qs_dir/src/Makefile
sed -i '/CXX = /s/^/#/g' $qs_dir/src/Makefile
sed -i '/CPPFLAGS = -DHAVE_MPI -DHAVE_HIP -x hip --offload-arch=gfx90a -fgpu-rdc -Wno-unused-result/s/^/#/g' $qs_dir/src/Makefile
sed -i '/LDFLAGS = -fgpu-rdc --hip-link --offload-arch=gfx90a/s/^/#/g' $qs_dir/src/Makefile

#Uncomment lines
sed -i '/OPENMP_FLAGS = -DHAVE_OPENMP -fopenmp $/s/^#//g' $qs_dir/src/Makefile
sed -i '/OPENMP_LDFLAGS = -fopenmp$/s/^#//g' $qs_dir/src/Makefile
sed -i '/OPTFLAGS = -g -O2$/s/^#//g' $qs_dir/src/Makefile
sed -i '/CXX=g++$/s/^#//g' $qs_dir/src/Makefile
sed -i '/CXXFLAGS = -std=c++11  $(OPTFLAGS) -Wpedantic$/s/^#//g' $qs_dir/src/Makefile
sed -i '/CPPFLAGS = $(OPENMP_FLAGS)$/s/^#//g' $qs_dir/src/Makefile
sed -i '/LDFLAGS = $(OPENMP_LDFLAGS) $/s/^#//g' $qs_dir/src/Makefile
cd $qs_dir/src
make

Cloning into 'Quicksilver'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 595 (delta 224), reused 191 (delta 188), pack-reused 307
Receiving objects: 100% (595/595), 327.26 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Rebuilding dependencies...
make[1]: *** [Makefile:365: depend] Error 127
echo "#define GIT_HASH \""eb68bb8d6fc53de1f65011d4e79ff2ed0dd60f3b"\" "> git_hash.hh
echo "#define GIT_VERS \""2023-Aug-18-15:12:10"\" "> git_vers.hh
g++ -DHAVE_OPENMP -fopenmp  -std=c++11  -g -O2 -Wpedantic -c CollisionEvent.cc -o CollisionEvent.o
g++ -DHAVE_OPENMP -fopenmp  -std=c++11  -g -O2 -Wpedantic -c CoralBenchmark.cc -o CoralBenchmark.o
g++ -DHAVE_OPENMP -fopenmp  -std=c++11  -g -O2 -Wpedantic -c CycleTracking.cc -o CycleTracking.o
g++ -DHAVE_OPENMP -fopenmp  -std=c++11  -g -O2 -Wpedantic -c DecompositionObject.cc -o DecompositionObject.o
g++ -DHAVE_OPENMP -fopenmp  -

In [3]:
#Step 2
module load vtune

Loading vtune version 2023.0.0
>


In [4]:
#Step 3
cd $nb_dir
git clone https://github.com/lanl/gs_patterns.git
cd gs_patterns
mkdir build
cd build
cmake ..
make

#Pin tool setup
cd ..
wget https://software.intel.com/sites/landingpage/pintool/downloads/pin-3.28-98749-g6643ecee5-gcc-linux.tar.gz
tar zxvf pin-3.28-98749-g6643ecee5-gcc-linux.tar.gz
export pin_dir="$gspatterns_dir/pin-3.28-98749-g6643ecee5-gcc-linux"
cp -rv pin_tracing/ImemROI $pin_dir/source/tools
cd $pin_dir/source/tools
sed -i '/Tests ToolUnitTests/ s/$/ ImemROI/' makefile
make -j

Cloning into 'gs_patterns'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 95 (delta 49), reused 30 (delta 11), pack-reused 0
Receiving objects: 100% (95/95), 39.18 KiB | 2.80 MiB/s, done.
Resolving deltas: 100% (49/49), done.
-- The C compiler identification is GNU 8.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done (0.2s)
-- Generating done (0.0s)
-- Build files have been written to: /nethome/jvalverde6/spatter/notebooks/gs_patterns/build
[ 50%] Building C object CMakeFiles/gs_patterns.dir/gs_patterns.c.o
[100%] Linking C executable gs_patterns
[100%] Built target gs_patterns
--2024-04-04 20:05:29--  https://software.intel.com/sites/landingpage/pintool/downloads/pin-3.28-98749-g6643ecee5-gcc-linux.tar.gz
Resolvin

pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/protection_test_tool.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/attach_app_restrict_memory.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/makefile
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/mitigation_policy_app.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/dcache_xscale_config.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/check_memory_access_tool.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/memory_allocation_from_app_access_protection_tool.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/new_delete_app.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/README
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/pin_stack_overflow.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memory/restrict_memory_vdso.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Memor

pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/images_on_attach_tool.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/split_image_mac.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/split_image_mac_app.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/makefile
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/full_path_app.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/load_dummy_dll.c
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/img_open_win.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/single_segment_app.s
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/img_find_by_addr.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/single_segment.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/images_on_attach_app.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/ImageTests/imageUnload_tool.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source

pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/run_many_win.bat
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/set_xmm_scratches_ia32.asm
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/threadlib_unix.c
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/movdir64b_check.c
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/supports_avx_intel64.asm
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/avx_check_ia32.s
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/hello.c
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/supports_amx_intel64.s
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/supports_avx512f_ia32.asm
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/makefile
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/testMacOsVersion
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/compare.py
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/Utils/avx_check_inte

pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/instrumentation_order19.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/instrumentation_order22.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/special_case3.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/instrumentation_order18.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/instrumentation_order11.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/redblue.s
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/special_case1.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/redblue.cpp
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/instrumentation_order16.ref
pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/InstrumentationOrderAndVersion/in

...

g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/change_mask.so obj-intel64/change_mask.o  -L../../../intel64/runtime/pincrt -L../../../intel64/lib -L../../../intel64/lib-ext -L../../../extras/xed-intel64/lib -lpin -lxed ../../../intel64/runtime/pincrt/crtendS.o -lpindwarf  -ldl-dynamic -nostdlib -lc++ -lc++abi -lm-dynamic -lc-dynamic -lunwind-dynamic
g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/threadOrder_tool.so obj-intel64/threadOrder_tool.o  -L../../../intel64/runtime/pincrt -L../../../intel64/lib -L../../../intel64/lib-ext -L../../../extras/xed-intel64/lib -lpin -lxed ../../../intel64/runtime/pincrt/crtendS.o -lpindwarf  -ldl-dynamic -nostdlib -lc++ -lc++abi -lm-dynamic -lc-dynamic -lunwind-dynamic
g++ 

g++  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/simple-pindb.exe simple-pindb.cpp obj-intel64/simple-pindb-asm-intel64.o     -Wl,--as-needed -lm -ldl -lpthread -g
cc  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/simple.exe simple.c    -Wl,--as-needed -lm -ldl -lpthread -g
cc  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/simple-static.exe simple.c    -static -Wl,--as-needed -lm -ldl -lpthread -g
g++  -DTARGET_IA32E -DHOST_IA32E 

g++  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/pthread-bare-exitgroup.exe pthread-bare-exitgroup.cpp pthread-bare-exitgroup-asm-intel64.s    -Wl,--as-needed -lm -ldl -lpthread -g
g++ -I../../../extras/components/include  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/thread-unix.exe thread-unix.cpp    -Wl,--as-needed -lm -ldl -lpthread -g
g++  -DTARGET_IA32E -DHOST_IA32E -DFUND_TC_TARGETCPU=FUND_CPU_INTEL64 -DFUND_TC_HOSTCPU=FUND_CPU_INTEL64 -DTARGET_LINUX -DFUND_TC_TARGETOS=FUND_OS_LINUX -DFUND_TC_HOSTOS=FUND_OS_LINUX  -I../../../source/tools/Utils -O0  -g  -o obj-intel64/ymm-intel64.exe ymm.cpp ymm-asm-intel64.s

g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/reattachImageLoadCallback_tool.so obj-intel64/reattachImageLoadCallback_tool.o  -L../../../intel64/runtime/pincrt -L../../../intel64/lib -L../../../intel64/lib-ext -L../../../extras/xed-intel64/lib -lpin -lxed ../../../intel64/runtime/pincrt/crtendS.o -lpindwarf  -ldl-dynamic -nostdlib -lc++ -lc++abi -lm-dynamic -lc-dynamic -lunwind-dynamic
g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/img_open_debug_info.so obj-intel64/img_open_debug_info.o  -L../../../intel64/runtime/pincrt -L../../../intel64/lib -L../../../intel64/lib-ext -L../../../extras/xed-intel64/lib -lpin -lxed ../../../intel64/runtime/pincrt/crtendS.o -lpindwarf  -ldl-dynamic -nostdlib -lc++ -lc++abi -

g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/iarg_reg_reference.so obj-intel64/iarg_reg_reference.o obj-intel64/regvalues.o obj-intel64/regvalue_test_utils.o obj-intel64/context_utils.o obj-intel64/register_modification_utils.o ../../../source/tools/Utils/obj-intel64/regvallib.a  -L../../../intel64/runtime/pincrt -L../../../intel64/lib -L../../../intel64/lib-ext -L../../../extras/xed-intel64/lib -lpin -lxed ../../../intel64/runtime/pincrt/crtendS.o -lpindwarf  -ldl-dynamic -nostdlib -lc++ -lc++abi -lm-dynamic -lc-dynamic -lunwind-dynamic
g++ -shared -Wl,--hash-style=sysv ../../../intel64/runtime/pincrt/crtbeginS.o -Wl,-Bsymbolic -Wl,--version-script=../../../source/include/pin/pintool.ver -fabi-version=2    -o obj-intel64/iarg_reg_reference_flags.so obj-intel64/iarg_reg_reference_flags.o  -L../../../intel64/runtime/pincrt -L../../../intel64/lib 

### Identifying Hotspots

In order to identify memory hot spots, we will need to profile an execution of the application.

In [5]:
module load vtune
vtune -collect hotspots "$qs_dir"/src/qs "$qs_dir"/Examples/AllAbsorb/allAbsorb.inp

vtune: Collection started. To stop the collection, either press CTRL-C or enter from another console window: vtune -r /nethome/jvalverde6/spatter/notebooks/gs_patterns/pin-3.28-98749-g6643ecee5-gcc-linux/source/tools/r000hs -command stop.
Copyright (c) 2016
Lawrence Livermore National Security, LLC
All Rights Reserved
Quicksilver Version     : 2023-Aug-18-15:12:10
Quicksilver Git Hash    : eb68bb8d6fc53de1f65011d4e79ff2ed0dd60f3b
MPI Version             : 3.0
Number of MPI ranks     : 1
Number of OpenMP Threads: 64
Number of OpenMP CPUs   : 64

Simulation:
   dt: 1e-08
   fMax: 0.1
   inputFile: 
   energySpectrum: 
   boundaryCondition: reflect
   loadBalance: 0
   cycleTimers: 0
   debugThreads: 0
   lx: 100
   ly: 100
   lz: 100
   nParticles: 1000000
   batchSize: 0
   nBatches: 10
   nSteps: 10
   nx: 10
   ny: 10
   nz: 10
   seed: 1029384756
   xDom: 0
   yDom: 0
   zDom: 0
   eMax: 20
   eMin: 1e-09
   nGroups: 230
   lowWeightCutoff: 0.001
   bTally: 1
   fTally: 1
   cTally: 

    Collector Type: Driverless Perf per-process counting,User-mode sampling and tracing
    CPU
        Name: Intel(R) Xeon(R) Processor code named Icelake
        Frequency: 2.000 GHz
        Logical CPU Count: 64
        LLC size: 50.3 MB 
        Cache Allocation Technology
            Level 2 capability: not detected
            Level 3 capability: available

If you want to skip descriptions of detected performance issues in the report,
enter: vtune -report summary -report-knob show-issues=false -r <my_result_dir>.
Alternatively, you may view the report in the csv format: vtune -report
<report_name> -format=csv.
vtune: Executing actions 100 % done                                            


Hotspot functions will be towards the end of the output, under **Top Hotspots**. 

Vtune might output some hotspot functions that look like:

1) func@0x1de24  
2) func@0x1dfd4

These are functions from an external library which are not traceable, therefore we will not focus on them. 

Functions that are traceable will usually have a friendlier name. We will trace the function **CycleTrackingFunction**.

Start by creating a file called **roi_funcs.txt**. This file should be located in the same directory where you call gs_patterns, and will contain the names of the functions you want to trace in your application, each on a separate line. 

In [6]:
cd $gspatterns_dir
touch roi_funcs.txt
echo "CycleTrackingFunction" > roi_funcs.txt

## Generating Traces

### Running Pin

Be advised running Pin might take a significant amount of time depending on the run time of your app, and how many functions you trace. 

In [7]:
$pin_dir/pin -t $pin_dir/source/tools/ImemROI/obj-intel64/ImemROIThreads.so -- "$qs_dir"/src/qs "$qs_dir"/Examples/AllAbsorb/allAbsorb.inp

PIN -- ROI_FUNC[ 0]: CycleTrackingFunction

Copyright (c) 2016
Lawrence Livermore National Security, LLC
All Rights Reserved
Quicksilver Version     : 2023-Aug-18-15:12:10
Quicksilver Git Hash    : eb68bb8d6fc53de1f65011d4e79ff2ed0dd60f3b
MPI Version             : 3.0
Number of MPI ranks     : 1
Number of OpenMP Threads: 64
Number of OpenMP CPUs   : 64

Simulation:
   dt: 1e-08
   fMax: 0.1
   inputFile: 
   energySpectrum: 
   boundaryCondition: reflect
   loadBalance: 0
   cycleTimers: 0
   debugThreads: 0
   lx: 100
   ly: 100
   lz: 100
   nParticles: 1000000
   batchSize: 0
   nBatches: 10
   nSteps: 10
   nx: 10
   ny: 10
   nz: 10
   seed: 1029384756
   xDom: 0
   yDom: 0
   zDom: 0
   eMax: 20
   eMin: 1e-09
   nGroups: 230
   lowWeightCutoff: 0.001
   bTally: 1
   fTally: 1
   cTally: 1
   coralBenchmark: 0
   crossSectionsOut:

Geometry:
   material: sourceMaterial
   shape: brick
   xMax: 100
   xMin: 0
   yMax: 100
   yMin: 0
   zMax: 100
   zMin: 0

Material:
   name: sour

There should be an output file called **roitrace.00.CycleTrackingFunction.bin**. For the next step, we must **gzip** the file. Again, this step might take a significant amount of time depending on the size of the file. 

In [8]:
gzip roitrace.00.CycleTrackingFunction.bin

### Running gs_patterns

This command will likely take the longest out of all. 

In [9]:
"$gspatterns_dir"/build/gs_patterns roitrace.00.CycleTrackingFunction.bin.gz "$qs_dir"/src/qs

First pass to find top gather / scatter iaddresses
....................................................................................................
 RESULTS 
DRTRACE STATS
DRTRACE LINES:              3505764658
OPCODES:                    2502302276
MEMOPCODES:                 1001861846
LOAD/STORES:                1003462374
OTHER:                               8

GATHER/SCATTER STATS: 
LOADS per GATHER:                6.278
STORES per SCATTER:              6.861
GATHER COUNT:                   24.773 (log2)
SCATTER COUNT:                  22.630 (log2)
OTHER  COUNT:                   29.537 (log2)

Symbol table lookup for gathers...done.
Symbol table lookup for scatters...done.

Second pass to fill gather / scatter subtraces


***************************************************************************************
roitrace.00.CycleTrackingFunction.bin.sbin
GIADDR   -- 0x41818c
SRCLINE  -- /nethome/jvalverde6/spatter/notebooks/Quicksilver/src/NuclearData.cc:193
GATHER % --  7.182% 

         2          2          4          4          4          4          4          4          4          0          0          0          0 
...
         8          8          8          8          8          8          8          8          8         10         10         10         10 
        10         10         10         10         10         12         12         12         12         12         12         12         12 
        12         14         14         14         14         14         14         14         14         14         16         16         16 

DIST HISTOGRAM --
   -18: 20873
   -16: 18502
   -14: 18426
   -12: 18115
   -10: 18371
    -8: 18557
    -6: 18339
    -4: 18384
    -2: 18502
     0: 7370616
     2: 849923
***************************************************************************************

***************************************************************************************
roitrace.00.CycleTrackingFunction.bin.sbin
GIADDR   -- 0x4128f4
SRC

GATHER % --  4.226% (512-bit chunks)
NDISTS  -- 5188344
         0          4          8         12         16         20         24         28         32         36         40         44         48 
        52         56         60         64         68         72         76         80         84         88         92          0          4 
         8         12         16         20         24         28         32         36         40         44         48         52         56 
...
17524368603790 17524368603794 17524368603798 17524368603802 17524368603806 17524368603810 17524368603814 17524368603818 17524368603822 17524368603826 17524368603830 17524368603834 17524368603838 
17524368603842 17524368603846 17524368603754 17524368603758 17524368603762 17524368603766 17524368603770 17524368603774 17524368603778 17524368603782 17524368603786 17524368603790 17524368603794 
17524368603798 17524368603802 17524368603806 17524368603810 17524368603814 17524368603818 17524368603822 17524368603

NDISTS  -- 108399
        46         46         46         46         46         46         46         46         46         46         46         46         46 
        46         46         46         46         46          0          0         46         46         46         46         46         46 
        46          0          0         46          0         46         46         46         46          0         46         46         46 
...
        46         46          0          0         46         46         46         46         46         46         46         46         46 
        46         46         46         46         46         46         46         46         46         46         46         46         46 
        46         46          0         46         46         46         46         46         46         46         46         46         46 

DIST HISTOGRAM --
   -46: 8704
     0: 90990
    46: 8704
*******************************************************

Once the above command is complete, you can run Spatter with the new **roitrace.00.CycleTrackingFunction.bin.json** file. Please refer to the [Getting Started](http://localhost:8080/notebooks/spatter/notebooks/GettingStarted.ipynb) notebook for more instructions on how to run Spatter. 

If you'd like to take a closer look at the pattern generated, please refer to the [Graphing Patterns]() notebook. 